In [82]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import time
import re
from tqdm.notebook import tqdm

In [83]:
import pandas as pd
import numpy as np
import random
from datetime import datetime ,timedelta , date

### __크롬 드라이버 연동__

In [84]:
my_path = 'D:/downloads/'
driver = webdriver.Chrome(my_path+'chromedriver.exe')
time.sleep(2)

__Load more 버튼을 cnt만큼 눌러서 모든 기사를 페이지에 로딩하는 함수__ 

In [88]:
def load_all_news(driver,cnt):
    while cnt:
        try:
            driver.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[2]/div/main/section/footer/div/a').click()
            time.sleep(1.5)
        except Exception as E:
            print(E)
            break
        cnt-=1
    return

__각 기사 태깅 후에 `12 hours ago` 와 같은 정보로 부터 날짜를 계산하는 함수:__  
- '~ago' 형식은 현재 시간에서 빼는 식으로  
- 'October 24' 형식은 그대로 적용

In [89]:
def get_date(tag): # 뉴스의 날짜 post date 계산
    global current_time
    
    posted_time_str = tag.select('div.info > header > div > div > span.time')[0].text
    
    if 'ago' in posted_time_str :
        
        time_diff = re.search('\d+',posted_time_str).group()
        
        if 'day' in posted_time_str:
            posted_time = current_time - timedelta(days = int(time_diff))
        elif 'hour' in posted_time_str:
            posted_time = current_time - timedelta(hours = int(time_diff))
        elif 'min' in posted_time_str:
            posted_time = current_time - timedelta(minutes = int(time_diff))
        else:
            print('error from 1')

    else:
        
        post_date = re.search('\d+',posted_time_str).group()
        
        if 'tober' in posted_time_str:
            posted_time = datetime(2020,10,int(post_date))
        elif 'tember' in posted_time_str:
            posted_time = datetime(2020,9,int(post_date))
        else:
            print('error from 2')
    
    return date.isoformat(posted_time)

### __검색 파라미터 설정__

In [90]:
current_time = datetime.now()
data=pd.DataFrame()
load_more = 5

### 메인 코드

In [91]:
driver.get('https://www.foxnews.com/category/politics/2020-presidential-election')
time.sleep(2)

load_all_news(driver,load_more)

html = driver.page_source
soup = BeautifulSoup(html,'lxml')
articles = soup.select('#wrapper > div.page > div.page-content > div > main > section > div > article')

news = [ {'headlines':article.select('div.info > header > h4 > a')[0].text,
         'date':get_date(article),
         'Press':'Fox',
         'Link':'https://www.foxnews.com'+article.select('div.info > header > h4 > a')[0].attrs['href'],
         'category':article.select('div.info > header > div > div > span.eyebrow > a')[0].text} for article in articles]
news_df = pd.DataFrame(news)
data = pd.concat([data,news_df])

In [92]:
data

,headlines,date,Press,Link,category
0,Trump reacts after Facebook removes 'NJ Women ...,2020-11-01,Fox,https://www.foxnews.com/tech/trump-reacts-afte...,Facebook
1,"Trump, Ernst take leads over Dem challengers i...",2020-11-01,Fox,https://www.foxnews.com/politics/trump-ernst-t...,Polls
2,'SNL' cold open has Kate McKinnon's Hillary Cl...,2020-11-01,Fox,https://www.foxnews.com/entertainment/snl-cold...,Saturday Night Live
3,"Live updates: Trump, Biden focus on crucial st...",2020-11-01,Fox,https://www.foxnews.com/politics/live-updates-...,2020 Presidential Election
4,Ivanka Trump breaks Obama record on way to $35...,2020-11-01,Fox,https://www.foxnews.com/politics/ivanka-trump-...,Fundraising
5,Biden focuses on slamming Trump coronavirus pl...,2020-11-01,Fox,https://www.foxnews.com/politics/slamming-trum...,Joe Biden
6,"Iowa farmers split between voting for Trump, B...",2020-11-01,Fox,https://www.foxnews.com/politics/iowa-farmers-...,Politics
7,Trump in Pennsylvania announces he signed exec...,2020-11-01,Fox,https://www.foxnews.com/politics/trump-pennsyl...,Donald Trump
8,Trump makes last-ditch pitch to Black American...,2020-11-01,Fox,https://www.foxnews.com/us/trump-makes-last-di...,2020 Presidential Election
9,Can Trump convince suburban women to back him?,2020-11-01,Fox,https://www.foxnews.com/politics/can-trump-con...,2020 Presidential Election


In [93]:
data.date.value_counts()

2020-10-31    39
2020-11-01    21
Name: date, dtype: int64

In [94]:
data.category.value_counts()

2020 Presidential Election    20
OPINION                        8
Fox News Flash                 4
Media                          4
Joe Biden                      3
ELECTIONS                      3
Polls                          2
Donald Trump                   2
Trump 2020 Campaign            1
Politics                       1
Texas                          1
North Carolina                 1
Saturday Night Live            1
Fox News Poll                  1
Wisconsin                      1
2020 House Races               1
Facebook                       1
New York City                  1
Hunter Biden                   1
Chicago                        1
Fundraising                    1
Politics on Late Night         1
Name: category, dtype: int64

### __수집된 링크들로부터 각각 본문 내용 수집__

In [95]:
body_text_list=[]
for url in data['Link']:

    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    body = soup.select('#wrapper > div.page-content > div.row.full > main > article > div > div.article-content > div.article-body')
    body_text = ''.join([p.text for p in body[0].select('p')])
    body_text_list.append(body_text)
    
data['text'] = pd.Series(body_text_list)

In [96]:
data.head(2)

,headlines,date,Press,Link,category,text
0,Trump reacts after Facebook removes 'NJ Women ...,2020-11-01,Fox,https://www.foxnews.com/tech/trump-reacts-afte...,Facebook,Greg Gutfeld on what Americans can look forwar...
1,"Trump, Ernst take leads over Dem challengers i...",2020-11-01,Fox,https://www.foxnews.com/politics/trump-ernst-t...,Polls,Fox News correspondent Garrett Tenney joins 'S...
